# Todays News Trends

### Program gets the current to headlines from 
* All News Media *
### and performs sentiment analysis on the news to identify the current sentiment.


In [1]:
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import time

import requests

from pprint import pprint

%matplotlib inline

# News API
import newsapi
from newsapi.newsapi_client import NewsApiClient
from newsapi.articles import Articles
from newsapi.sources import Sources

# NLTK
import nltk
from nltk import sent_tokenize
from string import punctuation

In [2]:
# Twitter API Keys
from config import (consumer_key, consumer_secret, access_token, access_token_secret)
#News API Keys
from config import newapi_key

In [3]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

#from config import (consumer_key, consumer_secret,
#                    access_token, access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# Init
newsapi = NewsApiClient(api_key='4b7e7bb7670e40f788f92667a6a2dd64')
newsapi_Articles = Articles(API_KEY=newapi_key)
newsapi_Sources = Sources(API_KEY=newapi_key)

In [4]:
# Get the top news newsapi, in the US, english, all categories.
def getCurrentHeadlines(sources='ALL'):
    top_headlines = newsapi.get_top_headlines(q=None, 
                                              sources=None, 
                                              language='en', 
                                              country=None, 
                                              category=None, 
                                              page_size=25, 
                                              page=None)
    
    #print(f"Total Results {top_headlines['totalResults']}")
    return top_headlines

In [5]:
#titles = [article['title'] for article in top_headlines['articles']]
headlines = getCurrentHeadlines()
#pprint(headlines)

In [6]:
len(headlines['articles'])

25

In [7]:
news_top_headlines_df = pd.DataFrame.from_dict(headlines['articles'])

In [8]:
news_top_headlines_df

,author,description,publishedAt,source,title,url,urlToImage
0,None,Activists hope that Riyadh will take greater s...,2018-06-28T04:37:00Z,"{'id': 'the-jerusalem-post', 'name': 'The Jeru...",Saudi Women Celebrate New Life Behind The Wheel,https://www.jpost.com/Middle-East/Saudi-Women-...,"https://images.jpost.com/image/upload/f_auto,f..."
1,The Hindu Net Desk,9.00 amBad weather halts Amarnath YatraBad wea...,2018-06-28T03:45:17.6100116Z,"{'id': 'the-hindu', 'name': 'The Hindu'}","Amarnath Yatra halted, Mexico poll campaign, a...",https://www.thehindu.com/news/news-in-pictures...,https://www.thehindu.com/news/o2ij1z/article24...
2,Catherine Shu,DVD and Blu-ray rental kiosk operator Redbox a...,2018-06-28T03:43:45Z,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Redbox lands deal with Warner to rent DVDs on ...,https://techcrunch.com/2018/06/27/redbox-lands...,https://techcrunch.com/wp-content/uploads/2018...
3,By Lara Keay For Mailonline,As most Brazilians celebrated sailing through ...,2018-06-28T03:41:03+00:00,"{'id': 'daily-mail', 'name': 'Daily Mail'}",Violence hits World Cup with fight inside stad...,http://www.dailymail.co.uk/news/article-589432...,http://i.dailymail.co.uk/i/newpix/2018/06/28/0...
4,None,African asylum camps idea plagued by doubts ov...,2018-06-28T03:37:57.6811638Z,"{'id': 'financial-times', 'name': 'Financial T...",Europe plots disembarkation scheme to keep mig...,https://www.ft.com/content/1e53958a-7a23-11e8-...,https://www.ft.com/__origami/service/image/v2/...
5,Fox News,Judge Napolitano's Chambers: Judge Andrew Napo...,2018-06-28T03:37:55.6497552Z,"{'id': 'fox-news', 'name': 'Fox News'}",Napolitano: SCOTUS hits hard against public-se...,http://video.foxnews.com/v/5802790600001/,http://a57.foxnews.com/media2.foxnews.com/Brig...
6,,Ticketing group says advice is a ‘precaution’ ...,2018-06-28T03:37:34Z,"{'id': 'the-guardian-au', 'name': 'The Guardia...",Ticketmaster hack: Australian customers urged ...,https://www.theguardian.com/money/2018/jun/28/...,https://i.guim.co.uk/img/media/8db9d56c27d23fa...
7,Gulnaz Khan,The geographically isolated Tiwi people have p...,2018-06-28T03:37:12.2022524Z,"{'id': 'national-geographic', 'name': 'Nationa...",Dreamlike Photos of Australia's Unspoiled Para...,https://www.nationalgeographic.com/travel/dest...,https://www.nationalgeographic.com/content/dam...
8,Karen Gardiner,From the spectacular Eiffel Tower display to T...,2018-06-28T03:37:11.7022151Z,"{'id': 'national-geographic', 'name': 'Nationa...",25 Dazzling Fireworks Pictures,https://www.nationalgeographic.com/travel/list...,https://www.nationalgeographic.com/content/dam...
9,Heather Brady,The group calling themselves 'Dare' says they ...,2018-06-28T03:37:10.8583964Z,"{'id': 'national-geographic', 'name': 'Nationa...","Amid a Civil War, These Syrians Keep Dancing",https://news.nationalgeographic.com/2018/06/da...,https://pmdvod.nationalgeographic.com/NG_Video...


In [9]:
news_channel_name_list = [article['source']['name'] for article in headlines['articles']]

In [10]:
#news_channel_name_list

In [11]:
news_top_headlines_df['News Channel Name'] = news_channel_name_list

In [12]:
news_top_headlines_df

,author,description,publishedAt,source,title,url,urlToImage,News Channel Name
0,None,Activists hope that Riyadh will take greater s...,2018-06-28T04:37:00Z,"{'id': 'the-jerusalem-post', 'name': 'The Jeru...",Saudi Women Celebrate New Life Behind The Wheel,https://www.jpost.com/Middle-East/Saudi-Women-...,"https://images.jpost.com/image/upload/f_auto,f...",The Jerusalem Post
1,The Hindu Net Desk,9.00 amBad weather halts Amarnath YatraBad wea...,2018-06-28T03:45:17.6100116Z,"{'id': 'the-hindu', 'name': 'The Hindu'}","Amarnath Yatra halted, Mexico poll campaign, a...",https://www.thehindu.com/news/news-in-pictures...,https://www.thehindu.com/news/o2ij1z/article24...,The Hindu
2,Catherine Shu,DVD and Blu-ray rental kiosk operator Redbox a...,2018-06-28T03:43:45Z,"{'id': 'techcrunch', 'name': 'TechCrunch'}",Redbox lands deal with Warner to rent DVDs on ...,https://techcrunch.com/2018/06/27/redbox-lands...,https://techcrunch.com/wp-content/uploads/2018...,TechCrunch
3,By Lara Keay For Mailonline,As most Brazilians celebrated sailing through ...,2018-06-28T03:41:03+00:00,"{'id': 'daily-mail', 'name': 'Daily Mail'}",Violence hits World Cup with fight inside stad...,http://www.dailymail.co.uk/news/article-589432...,http://i.dailymail.co.uk/i/newpix/2018/06/28/0...,Daily Mail
4,None,African asylum camps idea plagued by doubts ov...,2018-06-28T03:37:57.6811638Z,"{'id': 'financial-times', 'name': 'Financial T...",Europe plots disembarkation scheme to keep mig...,https://www.ft.com/content/1e53958a-7a23-11e8-...,https://www.ft.com/__origami/service/image/v2/...,Financial Times
5,Fox News,Judge Napolitano's Chambers: Judge Andrew Napo...,2018-06-28T03:37:55.6497552Z,"{'id': 'fox-news', 'name': 'Fox News'}",Napolitano: SCOTUS hits hard against public-se...,http://video.foxnews.com/v/5802790600001/,http://a57.foxnews.com/media2.foxnews.com/Brig...,Fox News
6,,Ticketing group says advice is a ‘precaution’ ...,2018-06-28T03:37:34Z,"{'id': 'the-guardian-au', 'name': 'The Guardia...",Ticketmaster hack: Australian customers urged ...,https://www.theguardian.com/money/2018/jun/28/...,https://i.guim.co.uk/img/media/8db9d56c27d23fa...,The Guardian (AU)
7,Gulnaz Khan,The geographically isolated Tiwi people have p...,2018-06-28T03:37:12.2022524Z,"{'id': 'national-geographic', 'name': 'Nationa...",Dreamlike Photos of Australia's Unspoiled Para...,https://www.nationalgeographic.com/travel/dest...,https://www.nationalgeographic.com/content/dam...,National Geographic
8,Karen Gardiner,From the spectacular Eiffel Tower display to T...,2018-06-28T03:37:11.7022151Z,"{'id': 'national-geographic', 'name': 'Nationa...",25 Dazzling Fireworks Pictures,https://www.nationalgeographic.com/travel/list...,https://www.nationalgeographic.com/content/dam...,National Geographic
9,Heather Brady,The group calling themselves 'Dare' says they ...,2018-06-28T03:37:10.8583964Z,"{'id': 'national-geographic', 'name': 'Nationa...","Amid a Civil War, These Syrians Keep Dancing",https://news.nationalgeographic.com/2018/06/da...,https://pmdvod.nationalgeographic.com/NG_Video...,National Geographic


In [ ]:
# remove the stopwords
remove_words = ['the', 'an', 'a', 'is', 'i', 'and', 'then', 'like', 'there', 'their']

In [ ]:
#remove the punctuation
def remove_punctuation(chars):
    return ''.join([c for c in chars if c not in punctuation])

# The DataFrame is 'news_top_headlines_df'
df = news_top_headlines_df

In [ ]:
#import our dataframe

df =

In [ ]:
#tokenize the text within the dataframe
df.loc[:, 'text_split'] = df.text.map(sent_tokenize)

In [ ]:
#get the text ready to analyze
sentences = []
for _, r in df.iterrows():
    for s in r("Headline").text_split:
        filtered_words = [remove_punctuation(w) for w in s.split() if w.lower() not in remove_words]
        # or using nltk.word_tokenize
        # filtered_words = [w for w in word_tokenize(s) if w.lower() not in remove_words and w not in punctuation]
        sentences.append({'sent_id': r.sent_id, 
                          'text': s.strip('.'), 
                          'words': filtered_words})
    wordfrq = r("Headline").text_split()

In [ ]:
#masterlist
masterlist = []

for channel in df:
    masterlist.append({'Channel': wordfrq})

In [ ]:
#rename the dataframe
df_words = pd.DataFrame(sentences)

In [ ]:
#word_cloud
#word freq
freq = nltk.FreqDist(wordfrq)
#tfidf
#stemming
#limitization
#thematic analysis

In [ ]:
#start the vader analysis
#run the vader on the orginal text
#import vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()